In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
import warnings
warnings.filterwarnings('ignore')
from qiskit import *
import time
from qiskit.quantum_info import SparsePauliOp
from qiskit.providers.fake_provider import *
from qiskit_aer.primitives import Estimator
import qiskit_aer.noise as noise
import pickle
from utils import *
from mitiq import zne

Uncomment code below if running noise model 1!

In [ ]:
id = 1
circuit = QuantumCircuit.from_qasm_file('circuit_1')
shot = 6000
noise_factor = range(1, 6, 2)
with open('fakekolkata.pkl', 'rb') as file:
    noise_model = pickle.load(file)
noise_model1 = noise.NoiseModel()
noise_model = noise_model1.from_dict(noise_model)
circuit.draw('mpl')

Uncomment code below if running noise model 2!

In [ ]:
# id = 2
# circuit = QuantumCircuit.from_qasm_file('circuit_2')
# shot = 6000
# noise_factor = range(1, 10, 2)
# with open('fakemontreal.pkl', 'rb') as file:
#     noise_model = pickle.load(file)
# noise_model1 = noise.NoiseModel()
# noise_model = noise_model1.from_dict(noise_model)
# circuit.draw('mpl')

Uncomment code below if running noise model 3!

In [ ]:
id = 3
circuit = QuantumCircuit.from_qasm_file('circuit_3')
shot = 1000
noise_factor = range(3, 8, 2)
with open('fakecairo.pkl', 'rb') as file:
    noise_model = pickle.load(file)
noise_model1 = noise.NoiseModel()
noise_model = noise_model1.from_dict(noise_model)
circuit.draw('mpl')

Load Hamilton of OH. The observable file is generated through Hamiltonian/gen_hamiltonian.py

In [ ]:
with open('Hamiltonian/observable', 'rb') as file:
        [op_list, coeffs] = pickle.load(file)

Set seed

In [ ]:
seeds = [20, 21, 30, 33, 36, 42, 43, 55, 67, 170]
seed = seeds[0]

Mapping logic qubits and Change the observable order accroding to the mapping 

In [ ]:
system_model = FakeMontreal()
transpiled_circuit = transpile(circuit, backend=system_model, seed_transpiler = seed)

res=transpiled_circuit.layout.initial_layout.get_virtual_bits()
maptable={}
for s in res:    
    if 'ancilla' not in s.register.name:
        maptable[s.index] = res[s]

# generate the observable 
labels= list(map(lambda s:rerange(s,maptable),op_list))
observable = SparsePauliOp(labels, coeffs)

Save transpiled circuit in QASM

In [ ]:
import qiskit.qasm3 as qasm3
with open('transpiled_circuit_{}.qasm'.format(id), 'w') as f:
    qasm3.dump(transpiled_circuit, f)

Pauli grouping: 631 pauli lists are patitioned into 39 commuting groups

In [ ]:
# group observables using Qiskit built-it function group_commuting()
grouped_observable = observable.group_commuting()    

Fold transpiled_circuit by the noise_factor, if the number of noise_factor is n, then n circuits will be generated.

In [ ]:
from zne.noise_amplification import LocalFoldingAmplifier
    
local_amplifier = LocalFoldingAmplifier()
circuits = []     
n = 3
for i in noise_factor:
    noisy_circuit = local_amplifier.amplify_circuit_noise(circuit=transpiled_circuit, noise_factor=i)    
    circuits.append(noisy_circuit)     

In [ ]:
estimator = Estimator(
        backend_options = {
            'method': 'statevector',
            'device': 'CPU',
            'noise_model': noise_model
        },
        run_options = {
            'shots': shot,
            'seed': seed,
        },
        skip_transpilation=True
    )    

Simulate all folding circuits

In [ ]:
s = time.time()
results = []

for i in range(len(noise_factor)):
    result = 0
    for num_group in range(len(grouped_observable)):
        job = estimator.run(circuits[i], grouped_observable[num_group])        
        result_values = job.result().values[0]        
        result += result_values
    results.append(result)
e = time.time()
print('Time:', e-s)

AvS extrapolation method is applied for noise model 1 and 3, otherwise Richardson method will be used

In [ ]:
if id == 1 or id == 3:
    result = avs(results)
else:
    result = zne.RichardsonFactory.extrapolate(noise_factor, results)

In [ ]:
print('Noise model {}'.format(id))
print('seed:', seed)
print('Energy:', result)